In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [14]:
#Load scaler picke file, one hot encoder and model
model = load_model('model.h5')
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
with open('one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder = pickle.load(file)       

In [15]:
input_data = {
    'CreditScore': 700,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [16]:
one_hot_enc = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
one_hot_enc_df = pd.DataFrame(one_hot_enc, columns=one_hot_encoder.get_feature_names_out(['Geography']))
one_hot_enc_df


/Library/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [37]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,Male,40,5,50000,2,1,1,60000


In [38]:
input_df['Gender'] = label_encoder.fit_transform(input_df['Gender'])

In [39]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,France,0,40,5,50000,2,1,1,60000


In [40]:
input_df = pd.concat([input_df.drop("Geography", axis=1), one_hot_enc_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,0,40,5,50000,2,1,1,60000,1.0,0.0,0.0


In [42]:
input_data_scaled1 = scaler.fit_transform(input_df)
input_data_scaled1

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [43]:
prediction = model.predict(input_data_scaled1)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.22470191]], dtype=float32)

In [45]:
prediction_proba = prediction[0][0]
prediction_proba

0.22470191

In [46]:
if prediction_proba > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.


In [49]:
label_encoder.classes_

array(['Male'], dtype=object)